In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/bbva-data-challenge-2023/bbva-data-challenge-2023/digital.csv
/kaggle/input/bbva-data-challenge-2023/bbva-data-challenge-2023/Diccionario de datos.xlsx
/kaggle/input/bbva-data-challenge-2023/bbva-data-challenge-2023/archive/sample_submission.csv
/kaggle/input/bbva-data-challenge-2023/bbva-data-challenge-2023/archive/balances.csv
/kaggle/input/bbva-data-challenge-2023/bbva-data-challenge-2023/archive/universe_train.csv
/kaggle/input/bbva-data-challenge-2023/bbva-data-challenge-2023/archive/movements.csv
/kaggle/input/bbva-data-challenge-2023/bbva-data-challenge-2023/archive/liabilities.csv
/kaggle/input/bbva-data-challenge-2023/bbva-data-challenge-2023/archive/universe_test.csv
/kaggle/input/bbva-data-challenge-2023/bbva-data-challenge-2023/archive/customers.csv


In [2]:
import dask.dataframe as dd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
from scipy import stats
import gc


In [3]:
! pip install duckdb=='0.9.2'
import duckdb

  Obtaining dependency information for duckdb==0.9.2 from https://files.pythonhosted.org/packages/7c/d5/7688cdc1b57c9ada17b6f6143a923606294b462e4b15e3ac249c04649bde/duckdb-0.9.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 45.6 MB/s eta 0:00:00:00:0100:01


In [4]:
%%time
path0 = '/kaggle/input/bbva-data-challenge-2023/bbva-data-challenge-2023/archive/'
train = pd.read_csv(f'{path0}universe_train.csv')
test = pd.read_csv(f'{path0}universe_test.csv')
test['attrition'] = -1
universe = pd.concat([train , test[train.columns]])#[['ID', 'period']]
print(universe.period.value_counts(1).sort_index())
print(train.shape, test.shape, universe.shape)

universe_path = 'universe.csv'  #tabla
universe.to_csv( universe_path , sep  =',', index = False)
universe.head()

period
period_1    0.106263
period_2    0.107940
period_3    0.109931
period_4    0.112305
period_5    0.113070
period_6    0.112493
period_7    0.111557
period_8    0.111922
period_9    0.114519
Name: proportion, dtype: float64
(535943, 3) (273636, 3) (809579, 3)
(535943, 3) (273636, 3) (809579, 3)
CPU times: user 4.05 s, sys: 402 ms, total: 4.45 s
Wall time: 6.39 s


,ID,attrition,period
0,4d5a33701cb5b30b0b0a924d80de4ae78fbd0b54e2117f...,0,period_5
1,bf7092e5f394d266143dfe90b3fc73eac51f0b0084d7a3...,0,period_5
2,270d8f8f607d19886c50edb7746c3670194134b56d31e2...,0,period_5
3,5e6b7bea5e4911329669f45728d3398ad54dfe11fbb16d...,0,period_5
4,98954adf775b9fce1c9e311a025ec3e0a1c6e90f991ef7...,0,period_5


In [5]:
digital = '/kaggle/input/bbva-data-challenge-2023/bbva-data-challenge-2023/digital.csv'
sample_submission = '/kaggle/input/bbva-data-challenge-2023/bbva-data-challenge-2023/archive/sample_submission.csv'
balances='/kaggle/input/bbva-data-challenge-2023/bbva-data-challenge-2023/archive/balances.csv'
universe_train = '/kaggle/input/bbva-data-challenge-2023/bbva-data-challenge-2023/archive/universe_train.csv'
movements = '/kaggle/input//bbva-data-challenge-2023/bbva-data-challenge-2023/archive/movements.csv'
liabilities = '/kaggle/input/bbva-data-challenge-2023/bbva-data-challenge-2023/archive/liabilities.csv'
universe_test = '/kaggle/input/bbva-data-challenge-2023/bbva-data-challenge-2023/archive/universe_test.csv'
customers ='/kaggle/input/bbva-data-challenge-2023/bbva-data-challenge-2023/archive/customers.csv'
universe_path = 'universe.csv'

# **UNIVERSO: PO**

In [6]:
PO = universe_path
periodo = 'period_6'

#OPERACIONES
operSUM= 'SUM'
operAVG = 'AVG'
operMAX= 'MAX'
operMIN= 'MIN'


# **FUENTE 1: Movements**

In [7]:
%%time

tabla = movements
alias = 'movements'
historia = 6
var1 = 'type_1'
var2 = 'type_2'
var3 = 'type_3'
var4 = 'type_4'

query1 = f""" with PO as ( 
select ID, period from '{PO}' 
where period = '{periodo}'  ),

tabla_h as (
select ID, period, 
{operSUM}( case when month>=12-{historia} + 1 then {var1} end) as {alias}_{operSUM}_{var1}_{historia},
{operSUM}( case when month>=12-{historia} + 1 then {var2} end) as {alias}_{operSUM}_{var2}_{historia},
{operSUM}( case when month>=12-{historia} + 1 then {var3} end) as {alias}_{operSUM}_{var3}_{historia},
{operSUM}( case when month>=12-{historia} + 1 then {var4} end) as {alias}_{operSUM}_{var4}_{historia},
{operAVG}( case when month>=12-{historia} + 1 then {var4} end) as {alias}_{operAVG}_{var4}_{historia}
from '{tabla}'
where period = '{periodo}'
group by 1,2)

SELECT 
    PO.ID, 
    PO.period,
    T1.{alias}_{operSUM}_{var1}_{historia}, 
    T1.{alias}_{operSUM}_{var2}_{historia}, 
    T1.{alias}_{operSUM}_{var3}_{historia}, 
    T1.{alias}_{operSUM}_{var4}_{historia}, 
    T1.{alias}_{operAVG}_{var4}_{historia}
from PO
LEFT JOIN tabla_h T1 ON T1.ID = PO.ID AND T1.period = PO.period 

"""
df1 = duckdb.query(query1).to_df()
print(df1.head())
df1.describe()


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

                                                  ID    period  \
0  b89bfe2cdfa61b59aca63ca326d5b7446c23308f95d339...  period_6   
1  23d5cc854528a6e07863cc1afad765d8b8613f35a35330...  period_6   
2  a234b2f7722d7f8ad1e37999a66b6b9310b83d4e130f73...  period_6   
3  46dcc263246afe069c3c96477f3b0110338aa3421f07a5...  period_6   
4  cc1ae29e103a569b4a9fe276c95aae056b22ccd616b561...  period_6   

   movements_SUM_type_1_6  movements_SUM_type_2_6  movements_SUM_type_3_6  \
0                 29.4701                 27.7808                  3.5201   
1                  8.9763                 13.1548                  3.1467   
2                  6.7504                 15.5535                  0.0000   
3                  5.4788                 26.3039                 12.5644   
4                 24.9075                  2.9660                 26.5680   

   movements_SUM_type_4_6  movements_AVG_type_4_6  
0                 23.1675                 3.86125  
1                 15.4188           

,movements_SUM_type_1_6,movements_SUM_type_2_6,movements_SUM_type_3_6,movements_SUM_type_4_6,movements_AVG_type_4_6
count,54767.000000,54767.000000,54767.000000,54767.000000,54767.000000
mean,7.956189,10.113795,5.925044,6.185738,1.296331
std,7.771930,8.515064,6.949166,8.353714,1.547488
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.604700,2.917650,0.000000,0.000000,0.000000
50%,5.507600,8.168900,3.521500,2.476300,0.555775
75%,12.830350,16.757850,9.270900,10.114500,2.500067
max,30.000000,30.000000,30.000000,30.000000,5.000000


# **FUENTE 2: Balances**

In [8]:
%%time

tabla = balances
alias = 'balances'
historia = 7
var1 = 'balance_amount'
var2 = 'days_default'
var3 = 'type_prod'

query2 = f""" with PO as ( 
select ID, period from '{PO}' 
where period = '{periodo}'  ),

tabla_int as (
select ID, period, 

( case 
    when type = 'type_3' and product = 'product_1'  then 't3p1' 
    when type = 'type_3' and product = 'product_2'  then 't3p2' 
    when type = 'type_5' and product = 'product_6'  then 't5p6' 
    when type = 'type_7' and product = 'product_1'  then 't7p2' 
    when type = 'type_7' and product = 'product_2'  then 't7p4' 
    when type = 'type_4' and product = 'product_4'  then 't4p2'
    when type = 'type_4' and product = 'product_2'  then 't4p4' 
    else 't12347_p24357'
    end
) as {var3}

from '{tabla}'
where period = '{periodo}'
group by 1,2,3),

tabla_h as (
select T_1.ID, T_1.period, 
{operSUM}( case when month>=12-{historia} + 1 then {var1} end) as {alias}_{operSUM}_{var1}_{historia},
{operSUM}( case when month>=12-{historia} + 1 then {var2} end) as {alias}_{operSUM}_{var2}_{historia},
{operMAX}( case when month>=12-{historia} + 1 then {var1} end) as {alias}_{operMAX}_{var1}_{historia},
{operMIN}( case when month>=12-{historia} + 1 then {var1} end) as {alias}_{operMIN}_{var1}_{historia},
( case when month>=12-{historia} + 1 then TI.{var3} end) as {alias}_{var3}_{historia}
from '{tabla}' T_1
INNER JOIN tabla_int TI ON TI.ID = T_1.ID AND TI.period = T_1.period 
where T_1.period = '{periodo}'
group by 1,2,7)


SELECT 
    PO.ID, 
    PO.period,
    T1.{alias}_{operSUM}_{var1}_{historia}, 
    T1.{alias}_{operSUM}_{var2}_{historia}, 
    T1.{alias}_{operMAX}_{var1}_{historia}, 
    T1.{alias}_{operMIN}_{var1}_{historia}, 
    T1.{alias}_{var3}_{historia}
from PO
LEFT JOIN tabla_h T1 ON T1.ID = PO.ID AND T1.period = PO.period 

"""
df2 = duckdb.query(query2).to_df()
print(df2.head())
df2.describe()


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

                                                  ID    period  \
0  521f9f4b8e69f3996e014aec7c9b89dd56d84009471476...  period_6   
1  5b3e00f6b11e7879cf3818bc15db735070ee17293acab1...  period_6   
2  dcd6f3f553a86d624d8d621ca912239a054bf2d6733d22...  period_6   
3  c1153b73377518670dc4db2daeeb650e5d5dd5c93e5679...  period_6   
4  ae8cb0cbbb0f5985e80b927d9619e1bb4a20008539a010...  period_6   

   balances_SUM_balance_amount_7  balances_SUM_days_default_7  \
0                            NaN                          NaN   
1                        37.9839                          0.0   
2                        32.5072                          0.0   
3                        83.0496                          0.0   
4                            NaN                          NaN   

   balances_MAX_balance_amount_7  balances_MIN_balance_amount_7  \
0                            NaN                            NaN   
1                         4.0889                         3.3667   
2          

,balances_SUM_balance_amount_7,balances_SUM_days_default_7,balances_MAX_balance_amount_7,balances_MIN_balance_amount_7
count,102191.000000,102191.000000,102191.000000,102191.000000
mean,64.038868,3.350898,4.168919,2.994779
std,36.796562,11.812410,0.530390,0.712035
min,0.149600,0.000000,0.149600,0.003000
25%,34.463150,0.000000,3.824350,2.601800
50%,58.921300,0.000000,4.191900,2.993900
75%,86.899100,0.000000,4.563400,3.421000
max,270.719500,188.419600,5.000000,5.000000


# **FUENTE 3: liabilities**

In [9]:
%%time

tabla = liabilities
alias = 'liabilities'
historia = 5
var1 = 'product_1'
var2 = 'product_2'

query3 = f""" with PO as ( 
select ID, period from '{PO}' 
where period = '{periodo}'  ),

tabla_h as (
select ID, period, 
{operSUM}( case when month>=12-{historia} + 1 then {var1} end) as {alias}_{operSUM}_{var1}_{historia},
{operSUM}( case when month>=12-{historia} + 1 then {var2} end) as {alias}_{operSUM}_{var2}_{historia},
{operAVG}( case when month>=12-{historia} + 1 then {var1} end) as {alias}_{operAVG}_{var1}_{historia},
{operAVG}( case when month>=12-{historia} + 1 then {var2} end) as {alias}_{operAVG}_{var2}_{historia},
{operMAX}( case when month>=12-{historia} + 1 then {var1} end) as {alias}_{operMAX}_{var1}_{historia}
from '{tabla}'
where period = '{periodo}'
group by 1,2)

SELECT 
    PO.ID, 
    PO.period,
    T1.{alias}_{operSUM}_{var1}_{historia}, 
    T1.{alias}_{operSUM}_{var2}_{historia}, 
    T1.{alias}_{operAVG}_{var1}_{historia}, 
    T1.{alias}_{operAVG}_{var2}_{historia}, 
    T1.{alias}_{operMAX}_{var1}_{historia}
from PO
LEFT JOIN tabla_h T1 ON T1.ID = PO.ID AND T1.period = PO.period 


"""
df3 = duckdb.query(query3).to_df()
print(df3.head())
df3.describe()


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

                                                  ID    period  \
0  0d8ca79126d02e579fbe3c1aaf5a7f3ecc2fd6bb296f26...  period_6   
1  17b495d8d2d89e7e8a7af0f6b0f277639039769cc63889...  period_6   
2  0a6314aaf97a449df3452eca5a303b915f1c79e117a963...  period_6   
3  7348961ff3db625cfd4f9a9bc7e876b72a25d37deb8248...  period_6   
4  5a9783ba1c907dd25be3f812568f5b06649c2b2e694cc5...  period_6   

   liabilities_SUM_product_1_5  liabilities_SUM_product_2_5  \
0                      16.3072                       0.0000   
1                      21.0769                       0.0000   
2                      11.7390                       6.6051   
3                      12.9648                       0.0000   
4                      19.9481                      20.1673   

   liabilities_AVG_product_1_5  liabilities_AVG_product_2_5  \
0                      3.26144                      0.00000   
1                      4.21538                      0.00000   
2                      2.34780     

,liabilities_SUM_product_1_5,liabilities_SUM_product_2_5,liabilities_AVG_product_1_5,liabilities_AVG_product_2_5,liabilities_MAX_product_1_5
count,88705.000000,88705.000000,88705.000000,88705.000000,88705.000000
mean,15.442857,3.237079,3.151459,0.651484,3.522792
std,5.594867,6.800924,1.071126,1.367134,1.028290
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,13.273700,0.000000,2.740880,0.000000,3.180400
50%,16.100400,0.000000,3.250320,0.000000,3.646500
75%,19.155000,0.020700,3.849260,0.004140,4.167000
max,25.000000,25.000000,5.000000,5.000000,5.000000


In [10]:
print(query2)

 with PO as ( 
select ID, period from 'universe.csv' 
where period = 'period_6'  ),

tabla_int as (
select ID, period, 

( case 
    when type = 'type_3' and product = 'product_1'  then 't3p1' 
    when type = 'type_3' and product = 'product_2'  then 't3p2' 
    when type = 'type_5' and product = 'product_6'  then 't5p6' 
    when type = 'type_7' and product = 'product_1'  then 't7p2' 
    when type = 'type_7' and product = 'product_2'  then 't7p4' 
    when type = 'type_4' and product = 'product_4'  then 't4p2'
    when type = 'type_4' and product = 'product_2'  then 't4p4' 
    else 't12347_p24357'
    end
) as type_prod

from '/kaggle/input/bbva-data-challenge-2023/bbva-data-challenge-2023/archive/balances.csv'
where period = 'period_6'
group by 1,2,3),

tabla_h as (
select T_1.ID, T_1.period, 
SUM( case when month>=12-7 + 1 then balance_amount end) as balances_SUM_balance_amount_7,
SUM( case when month>=12-7 + 1 then days_default end) as balances_SUM_days_default_7,
MAX( case when 

# **FUENTE 4: Digital**

In [11]:
%%time

tabla = digital
alias = 'digital'
historia = 8
var1 = 'dig_1'
var2 = 'dig_2'
var3 = 'dig_3'
var4 = 'dig_4'
var5 = 'dig_5'

query4 = f""" with PO as ( 
select ID, period from '{PO}' 
where period = '{periodo}'  ),

tabla_h as (
select ID, period, 
{operSUM}( case when month>=12-{historia} + 1 then {var1} end) as {alias}_{operSUM}_{var1}_{historia},
{operSUM}( case when month>=12-{historia} + 1 then {var2} end) as {alias}_{operSUM}_{var2}_{historia},
{operSUM}( case when month>=12-{historia} + 1 then {var3} end) as {alias}_{operSUM}_{var3}_{historia},
{operSUM}( case when month>=12-{historia} + 1 then {var4} end) as {alias}_{operSUM}_{var4}_{historia},
{operSUM}( case when month>=12-{historia} + 1 then {var5} end) as {alias}_{operSUM}_{var5}_{historia}
from '{tabla}'
where period = '{periodo}'
group by 1,2)

SELECT 
    PO.ID, 
    PO.period,
    T1.{alias}_{operSUM}_{var1}_{historia}, 
    T1.{alias}_{operSUM}_{var2}_{historia}, 
    T1.{alias}_{operSUM}_{var3}_{historia}, 
    T1.{alias}_{operSUM}_{var4}_{historia}, 
    T1.{alias}_{operSUM}_{var5}_{historia}
from PO
LEFT JOIN tabla_h T1 ON T1.ID = PO.ID AND T1.period = PO.period 

"""
df4 = duckdb.query(query4).to_df()
print(df4.head())
df4.describe()


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

                                                  ID    period  \
0  83bbb1215d3de0f9260af712df4aee90a481c87af0218f...  period_6   
1  77230ac4fb062ee011a676a3f977af9c8554ed51aa2eb6...  period_6   
2  92c7ed9063533fff455be290c30ac33d472c5c2775fb99...  period_6   
3  426eaf841ed02742c45042820a58f33983f5fefe68b39b...  period_6   
4  f014080c824589a330fc83a138cd61a2c30ee55c297855...  period_6   

   digital_SUM_dig_1_8  digital_SUM_dig_2_8  digital_SUM_dig_3_8  \
0               1.9343              24.1575              24.3479   
1               3.8971              24.1375              23.2917   
2               9.4091              31.5847              29.6434   
3              11.6050              25.4810              25.2270   
4               5.6770              23.5741              22.1715   

   digital_SUM_dig_4_8  digital_SUM_dig_5_8  
0              13.3174               3.5521  
1              19.7542              10.1081  
2              19.7408              10.5741  
3         

,digital_SUM_dig_1_8,digital_SUM_dig_2_8,digital_SUM_dig_3_8,digital_SUM_dig_4_8,digital_SUM_dig_5_8
count,71881.000000,71881.000000,71881.000000,71881.000000,71881.000000
mean,11.550043,25.042681,24.704274,18.009045,11.306528
std,6.642436,8.284966,7.755095,7.996471,6.829211
min,0.000000,1.045700,2.051400,0.000000,0.000000
25%,6.601200,21.694400,22.228900,14.574400,6.585800
50%,10.798900,27.480000,26.665100,19.553000,10.710100
75%,15.553000,30.768300,29.748400,23.469600,15.542400
max,40.000000,40.000000,40.000000,37.666800,39.270600


# **FUENTE 5: Customers**

In [12]:
%%time

tabla = customers
alias = 'customers'
var1 = 'age'
var2 = 'income'
var3 = 'payroll'
var4 = 'bureau_risk'
var5 = 'time_from_specialized'


query5 = f""" with PO as ( 
select ID from '{PO}' 
),

tabla_h as (
select ID, 
{var1} AS {alias}_{var1},
{var2} AS {alias}_{var2},
{var3} AS {alias}_{var3},
{var4} AS {alias}_{var4},
{var5} AS {alias}_{var5}
from '{tabla}'
)

SELECT 
    PO.ID, 
    T1.{alias}_{var1}, 
    T1.{alias}_{var2}, 
    T1.{alias}_{var3}, 
    T1.{alias}_{var4}, 
    T1.{alias}_{var5}
from PO
LEFT JOIN tabla_h T1 ON T1.ID = PO.ID 

"""
df5 = duckdb.query(query5).to_df()
print(df5.head())
df5.describe()


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

                                                  ID  customers_age  \
0  4d5a33701cb5b30b0b0a924d80de4ae78fbd0b54e2117f...         3.8498   
1  bf7092e5f394d266143dfe90b3fc73eac51f0b0084d7a3...         4.5476   
2  270d8f8f607d19886c50edb7746c3670194134b56d31e2...         4.3600   
3  5e6b7bea5e4911329669f45728d3398ad54dfe11fbb16d...         4.3914   
4  98954adf775b9fce1c9e311a025ec3e0a1c6e90f991ef7...         4.3077   

   customers_income  customers_payroll customers_bureau_risk  \
0            4.6527                0.0            category_6   
1            4.2447                0.0            category_4   
2            4.2117                0.0            category_9   
3            4.1584                0.0            category_2   
4            4.3285                0.0            category_6   

   customers_time_from_specialized  
0                            0.000  
1                            4.274  
2                            0.000  
3                            0.000  
4  

,customers_age,customers_income,customers_payroll,customers_time_from_specialized
count,809579.000000,809579.000000,809579.000000,809579.000000
mean,4.275201,4.284237,1.417533,1.333632
std,0.377544,0.477204,2.017392,1.946502
min,0.000000,0.000000,0.000000,0.000000
25%,4.001100,4.098200,0.000000,0.000000
50%,4.266700,4.286900,0.000000,0.000000
75%,4.549500,4.509800,3.973700,3.619300
max,5.000000,5.000000,5.000000,5.000000


In [13]:
x = df1.merge(df2, on = ['ID', 'period'], how = 'inner')
x = x.merge(df3, on = ['ID', 'period'], how = 'inner')
x = x.merge(df4, on = ['ID', 'period'], how = 'inner')
x = x.merge(df5, on = ['ID'], how = 'inner')
x

,ID,period,movements_SUM_type_1_6,movements_SUM_type_2_6,movements_SUM_type_3_6,movements_SUM_type_4_6,movements_AVG_type_4_6,balances_SUM_balance_amount_7,balances_SUM_days_default_7,balances_MAX_balance_amount_7,...,digital_SUM_dig_1_8,digital_SUM_dig_2_8,digital_SUM_dig_3_8,digital_SUM_dig_4_8,digital_SUM_dig_5_8,customers_age,customers_income,customers_payroll,customers_bureau_risk,customers_time_from_specialized
0,b89bfe2cdfa61b59aca63ca326d5b7446c23308f95d339...,period_6,29.4701,27.7808,3.5201,23.1675,3.86125,61.9643,0.0,5.0000,...,25.0349,35.1997,33.5785,17.4908,8.1168,4.2247,4.5166,0.000,category_2,4.1324
1,b89bfe2cdfa61b59aca63ca326d5b7446c23308f95d339...,period_6,29.4701,27.7808,3.5201,23.1675,3.86125,61.9643,0.0,5.0000,...,25.0349,35.1997,33.5785,17.4908,8.1168,4.2247,4.5166,0.000,category_2,4.1324
2,b89bfe2cdfa61b59aca63ca326d5b7446c23308f95d339...,period_6,29.4701,27.7808,3.5201,23.1675,3.86125,NaN,NaN,NaN,...,25.0349,35.1997,33.5785,17.4908,8.1168,4.2247,4.5166,0.000,category_2,4.1324
3,b89bfe2cdfa61b59aca63ca326d5b7446c23308f95d339...,period_6,29.4701,27.7808,3.5201,23.1675,3.86125,61.9643,0.0,5.0000,...,25.0349,35.1997,33.5785,17.4908,8.1168,4.2247,4.5166,0.000,category_2,4.1324
4,23d5cc854528a6e07863cc1afad765d8b8613f35a35330...,period_6,8.9763,13.1548,3.1467,15.4188,2.56980,93.1907,0.0,5.0000,...,20.3949,29.5340,27.3151,25.2359,14.0470,4.0625,4.2235,0.000,category_4,3.8784
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188067,24f1502e3ec9aaa485d0df6c929d5c1cbf922ba1d58647...,period_6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,6.9291,15.5814,14.0449,13.5564,9.1530,4.5227,4.1062,3.208,category_3,0.0000
188068,ea662237f56f2320ace073e6be50034e86a871a0041ff0...,period_6,NaN,NaN,NaN,NaN,NaN,8.9218,0.0,4.5456,...,5.0000,5.0000,5.0000,0.0000,0.0000,4.2949,3.8558,0.000,category_1,4.5150
188069,ea662237f56f2320ace073e6be50034e86a871a0041ff0...,period_6,NaN,NaN,NaN,NaN,NaN,8.9218,0.0,4.5456,...,5.0000,5.0000,5.0000,0.0000,0.0000,4.2949,3.8558,0.000,category_1,4.5150
188070,e7bc7726bfa5ef0529257f7d56b42296b61f609e99b98e...,period_6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,4.7753,4.0071,0.000,category_3,0.0000


In [18]:
universe.head(30)

,ID,attrition,period
0,4d5a33701cb5b30b0b0a924d80de4ae78fbd0b54e2117f...,0,period_5
1,bf7092e5f394d266143dfe90b3fc73eac51f0b0084d7a3...,0,period_5
2,270d8f8f607d19886c50edb7746c3670194134b56d31e2...,0,period_5
3,5e6b7bea5e4911329669f45728d3398ad54dfe11fbb16d...,0,period_5
4,98954adf775b9fce1c9e311a025ec3e0a1c6e90f991ef7...,0,period_5
5,56fe8d95f1af9103e92ce450ece54b4647ccb001a866e6...,1,period_5
6,f728012da429bb829188103c2cb56ef496a7b7e27360fd...,0,period_5
7,9bcc48234dd33f97a9e5d475d39fcb4d562bd7489404d1...,0,period_5
8,d8e3721b57230439133fd302f51312c6872eec574877a9...,0,period_5
9,0ca3a2a78e78f898860813a77bf853f49b4ce4a814da25...,0,period_5


In [20]:
%%time


queryf = f""" 
select * from '{PO}' 
where period = '{periodo}' 


"""
dff = duckdb.query(queryf).to_df()
print(dff.head())
dff.describe()

                                                  ID  attrition    period
0  521f9f4b8e69f3996e014aec7c9b89dd56d84009471476...          0  period_6
1  5b3e00f6b11e7879cf3818bc15db735070ee17293acab1...          0  period_6
2  dcd6f3f553a86d624d8d621ca912239a054bf2d6733d22...          1  period_6
3  c1153b73377518670dc4db2daeeb650e5d5dd5c93e5679...          0  period_6
4  349420da8b9637537e14c6b60af05e57be3bdc0dac446f...          0  period_6
CPU times: user 1.7 s, sys: 127 ms, total: 1.83 s
Wall time: 696 ms


,attrition
count,91072.000000
mean,0.288706
std,0.895147
min,0.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,5.000000


In [28]:
#para entrenamiento 
xf = x.merge(dff, on = ['ID', 'period'], how = 'inner')
xf.columns


Index(['ID', 'period', 'movements_SUM_type_1_6', 'movements_SUM_type_2_6',
       'movements_SUM_type_3_6', 'movements_SUM_type_4_6',
       'movements_AVG_type_4_6', 'balances_SUM_balance_amount_7',
       'balances_SUM_days_default_7', 'balances_MAX_balance_amount_7',
       'balances_MIN_balance_amount_7', 'balances_type_prod_7',
       'liabilities_SUM_product_1_5', 'liabilities_SUM_product_2_5',
       'liabilities_AVG_product_1_5', 'liabilities_AVG_product_2_5',
       'liabilities_MAX_product_1_5', 'digital_SUM_dig_1_8',
       'digital_SUM_dig_2_8', 'digital_SUM_dig_3_8', 'digital_SUM_dig_4_8',
       'digital_SUM_dig_5_8', 'customers_age', 'customers_income',
       'customers_payroll', 'customers_bureau_risk',
       'customers_time_from_specialized', 'attrition'],
      dtype='object')

In [51]:
Y = xf['attrition']
X = xf.drop(columns=['attrition', 'customers_bureau_risk', 'ID', 'period', 'balances_type_prod_7'])

In [52]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)


In [53]:
from sklearn.utils.class_weight import compute_class_weight
class_weights = compute_class_weight('balanced', classes=[0,1,2,3,4,5], y=y_train)
#lgb_train = lgb.Dataset(X_train, y_train, weight=class_weights[y_train])
class_weights#[y_train]

array([ 0.19550432,  2.00049196, 10.05863083, 18.64399009, 11.96953063,
        6.73185682])

In [43]:
!pip install lightgbm==3.3.2  
import lightgbm as lgb  #  lightgbm==3.3.2   sagemaker'4.1.0'
from sklearn.metrics import f1_score
import matplotlib.pyplot as plt


lgb.__version__

'3.3.2'

In [54]:
%%time
train_data_lgb = lgb.Dataset(X_train, label=y_train, weight=class_weights[y_train])
test_data_lgb = lgb.Dataset(X_test, label=y_test)

param = {
    'objective': 'multiclass',
    'boosting': 'gbdt',
    'metric': 'multi_logloss',
    ##'metric': 'multi_error',
    'task': 'train',
    'num_class': 6,
    "max_depth":6,
    "num_leaves":32,
    'learning_rate': 0.05,
    "min_child_samples": 100,
    'feature_fraction': 0.9,
    "bagging_freq":1,
    'bagging_fraction': 0.9,
    "lambda_l1":1,
    "lambda_l2":1,
    'verbosity': -1,
    'is_unbalance': True,
    'num_boost_round': 3000
}


model = lgb.train(
param,
train_data_lgb,
valid_sets=[test_data_lgb, train_data_lgb], 
verbose_eval=100,
early_stopping_rounds=500, 
)

/opt/conda/lib/python3.10/site-packages/lightgbm/engine.py:177: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
/opt/conda/lib/python3.10/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.10/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


Training until validation scores don't improve for 500 rounds
[100]	training's multi_logloss: 0.920538	valid_0's multi_logloss: 1.26657
[200]	training's multi_logloss: 0.724342	valid_0's multi_logloss: 1.13235
[300]	training's multi_logloss: 0.584784	valid_0's multi_logloss: 1.01952
[400]	training's multi_logloss: 0.485023	valid_0's multi_logloss: 0.928626
[500]	training's multi_logloss: 0.411875	valid_0's multi_logloss: 0.85155
[600]	training's multi_logloss: 0.354218	valid_0's multi_logloss: 0.785107
[700]	training's multi_logloss: 0.308053	valid_0's multi_logloss: 0.726647
[800]	training's multi_logloss: 0.271724	valid_0's multi_logloss: 0.677117
[900]	training's multi_logloss: 0.24155	valid_0's multi_logloss: 0.633669
[1000]	training's multi_logloss: 0.216755	valid_0's multi_logloss: 0.594934
[1100]	training's multi_logloss: 0.19603	valid_0's multi_logloss: 0.560521
[1200]	training's multi_logloss: 0.17834	valid_0's multi_logloss: 0.531073
[1300]	training's multi_logloss: 0.163163	

In [55]:
fi =pd.DataFrame(zip( model.feature_name(), model.feature_importance(importance_type='gain'))).sort_values(1,ascending = False)
fi.columns = ['variable', 'gain']

fi = fi.sort_values('gain', ascending = False)
fi['cumsum'] = fi.gain.cumsum()/fi.gain.sum()
fi['gain_porcentaje'] = (fi.gain/fi.gain.sum()*100).round(3)
colliab = list(fi[fi['cumsum']<.95].variable )
print(len(colliab), fi.shape)
print(colliab)
fi.head(50)

19 (23, 4)
['liabilities_MAX_product_1_5', 'liabilities_SUM_product_1_5', 'customers_income', 'liabilities_AVG_product_1_5', 'customers_age', 'digital_SUM_dig_2_8', 'digital_SUM_dig_5_8', 'digital_SUM_dig_1_8', 'digital_SUM_dig_3_8', 'digital_SUM_dig_4_8', 'movements_SUM_type_2_6', 'customers_time_from_specialized', 'movements_SUM_type_1_6', 'movements_SUM_type_3_6', 'balances_SUM_balance_amount_7', 'balances_MAX_balance_amount_7', 'balances_SUM_days_default_7', 'balances_MIN_balance_amount_7', 'customers_payroll']


,variable,gain,cumsum,gain_porcentaje
13,liabilities_MAX_product_1_5,788778.154317,0.160307,16.031
9,liabilities_SUM_product_1_5,373629.009176,0.236241,7.593
20,customers_income,364822.037225,0.310385,7.414
11,liabilities_AVG_product_1_5,335985.214001,0.378669,6.828
19,customers_age,299672.244711,0.439573,6.090
15,digital_SUM_dig_2_8,294775.846729,0.499481,5.991
18,digital_SUM_dig_5_8,251508.602250,0.550596,5.112
14,digital_SUM_dig_1_8,247132.180289,0.600822,5.023
16,digital_SUM_dig_3_8,228348.674518,0.647230,4.641
17,digital_SUM_dig_4_8,195884.067033,0.687041,3.981
